In [1]:
!pip install psycopg2-binary==2.9.3
import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.2 MB/s eta 0:00:0000:0100:01


In [2]:
from sqlalchemy import create_engine, text
import pandas as pd

# Подготовка эмбедингов для текстов
Можно использовать готовые предобученные модели из семейства Берта

In [3]:
## подключение, реальные доступы убраны из ячейки
conn_uri = "postgresql://логин:пароль@путь:порт"
connection = create_engine(conn_uri)

## таблица post
query_post = "SELECT * FROM post_text_df"

df_post = pd.read_sql_query(con=connection.connect(), 
                                  sql=text(query_post))

In [4]:
df_post

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


# Готовим датасет 

In [2]:
## выгружаем токенайзер и модель

from transformers import AutoTokenizer
from transformers import BertModel  # https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
from transformers import RobertaModel  # https://huggingface.co/docs/transformers/model_doc/roberta#transformers.RobertaModel
from transformers import DistilBertModel  # https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertModel


def get_model(model_name):
    assert model_name in ['bert', 'roberta', 'distilbert']

    checkpoint_names = {
        'bert': 'bert-base-cased',  # https://huggingface.co/bert-base-cased
        'roberta': 'roberta-base',  # https://huggingface.co/roberta-base
        'distilbert': 'distilbert-base-cased'  # https://huggingface.co/distilbert-base-cased
    }

    model_classes = {
        'bert': BertModel,
        'roberta': RobertaModel,
        'distilbert': DistilBertModel
    }

    return AutoTokenizer.from_pretrained(checkpoint_names[model_name]), model_classes[model_name].from_pretrained(checkpoint_names[model_name])

In [3]:
tokenizer, model = get_model('roberta')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_post[['text']])

In [32]:
## применим токенизацию
def tokenization(example):
    return tokenizer.batch_encode_plus(example['text'], add_special_tokens=True, return_token_type_ids=False, truncation=True)


dataset = dataset.map(tokenization, batched=True)

Map:   0%|          | 0/7023 [00:00<?, ? examples/s]

In [10]:
## проверим ключи
dataset[0].keys()

dict_keys(['text', 'input_ids', 'attention_mask'])

In [33]:
dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

In [12]:
dataset[0]

{'input_ids': tensor([  101,  1993,  4190,  4749,  1558, 11040,  1109,  1993,  5863,  4291,
          1209,  2760,  1106,  1339,  3021,  7806,  1166,  1103,  1397,  1160,
          1201,   117,  1103,  1418,  7018,  1104,  7873,   113,  3823,  1658,
           114,  1144,  1163,   119,  1109,  2114, 21081,  5980,  1104,  2557,
          1276, 15765,  1125,  3015,  1146,  1107,  1103,  1314,  1210,  1808,
          1104,  1516,  1106,  1147,  1436,  3001,  1107,  2022,  1201,   119,
          1109,  3606,  1338,  2693,  3670,  5600,  1217,  6024,  1112,   170,
          1558,  4517,   119,  1438,   117,  1103,  3823,  1658,  1276,  1103,
          2006,  1993,  4190,  1253,  3544,  1558, 11040,  1105,  6926,  1115,
          3213,  1110,  1383,  1106,  3345,   119,  1135,  3055, 24647,  2670,
          3213,  1209,  3345,  1121,  1167,  1190,   124,   110,  1107,  1516,
          1106,   170,  1376,  2071,   123,   119,   126,   110,  1107,  1241,
          1478,  1105,  1386,   119,  2

In [34]:
### добавление паддинга чтобы подавались посты все одного размера в модель 
from transformers import DataCollatorWithPadding


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [35]:
### готовим даталоудеры
from torch.utils.data import DataLoader


loader = DataLoader(dataset, batch_size=32, collate_fn=data_collator, pin_memory=True, shuffle=False)

In [15]:
loader.dataset[0]

{'input_ids': tensor([  101,  1993,  4190,  4749,  1558, 11040,  1109,  1993,  5863,  4291,
          1209,  2760,  1106,  1339,  3021,  7806,  1166,  1103,  1397,  1160,
          1201,   117,  1103,  1418,  7018,  1104,  7873,   113,  3823,  1658,
           114,  1144,  1163,   119,  1109,  2114, 21081,  5980,  1104,  2557,
          1276, 15765,  1125,  3015,  1146,  1107,  1103,  1314,  1210,  1808,
          1104,  1516,  1106,  1147,  1436,  3001,  1107,  2022,  1201,   119,
          1109,  3606,  1338,  2693,  3670,  5600,  1217,  6024,  1112,   170,
          1558,  4517,   119,  1438,   117,  1103,  3823,  1658,  1276,  1103,
          2006,  1993,  4190,  1253,  3544,  1558, 11040,  1105,  6926,  1115,
          3213,  1110,  1383,  1106,  3345,   119,  1135,  3055, 24647,  2670,
          3213,  1209,  3345,  1121,  1167,  1190,   124,   110,  1107,  1516,
          1106,   170,  1376,  2071,   123,   119,   126,   110,  1107,  1241,
          1478,  1105,  1386,   119,  2

# Выгрузка эмбедингов

In [26]:
import torch
from tqdm import tqdm


@torch.inference_mode()
def get_embeddings_labels(model, loader):
    model.eval()
    
    total_embeddings = []
    
    for batch in tqdm(loader):
        batch = {key: batch[key].to(device) for key in ['attention_mask', 'input_ids']}

        embeddings = model(**batch)['last_hidden_state'][:, 0, :]

        total_embeddings.append(embeddings.cpu())

    return torch.cat(total_embeddings, dim=0)

In [36]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)
print(torch.cuda.get_device_name())

model = model.to(device)

cuda:0
Tesla T4


In [37]:
embeddings = get_embeddings_labels(model, loader)

embeddings

100%|██████████| 220/220 [03:58<00:00,  1.08s/it]


tensor([[-0.0386,  0.0984,  0.0295,  ..., -0.0810, -0.0354, -0.0485],
        [-0.0079,  0.0890,  0.0422,  ..., -0.0788, -0.0243,  0.0108],
        [-0.0668,  0.1124,  0.0323,  ..., -0.1262, -0.0849, -0.0188],
        ...,
        [-0.0770,  0.0510, -0.0283,  ..., -0.1375, -0.0531, -0.0351],
        [-0.0743,  0.0876,  0.0101,  ..., -0.1202, -0.0096, -0.0402],
        [-0.1092,  0.0975, -0.0014,  ..., -0.0713,  0.0044,  0.0455]])

In [39]:
torch.save(embeddings, 'emb_posts_4.pt')